# Setup

In [1]:
%%time
1

CPU times: user 1 μs, sys: 1 μs, total: 2 μs
Wall time: 3.34 μs


1

In [2]:
# Required imports
from vina import Vina
import subprocess
from pathlib import Path
import rdkit
from rdkit import Chem
import py3Dmol
import pandas as pd
import os
import requests
import contextlib

In [3]:
from ipywidgets import interact, IntSlider,Dropdown
import ipywidgets, copy
from IPython.display import display, Markdown

In [4]:
import prody
prody.LOGGER.verbosity = 'none'

# Required Functions


In [5]:
def generate_receptor_file(proteinPDB,proteinFilename,box_size,box_center):
  subprocess.run([
      "mk_prepare_receptor.py",
      "-i", proteinPDB,
      "-o", proteinFilename,
      "-p",
      "-v",
      #"-g",
      "--box_size", str(box_size[0]), str(box_size[1]), str(box_size[2]),
      "--box_center", str(box_center[0]), str(box_center[1]), str(box_center[2])
      ], check=True,
    stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
  return

In [6]:
def translate_ligand_to_box_center(ligand_sdf, output_sdf, box_center):
    mol = Chem.SDMolSupplier(ligand_sdf, removeHs=False)[0]  # read first molecule
    conf = mol.GetConformer()

    # Calculate centroid
    num_atoms = mol.GetNumAtoms()
    centroid = [0.0, 0.0, 0.0]
    for i in range(num_atoms):
        pos = conf.GetAtomPosition(i)
        centroid[0] += pos.x
        centroid[1] += pos.y
        centroid[2] += pos.z
    centroid = [c / num_atoms for c in centroid]

    # Compute translation vector
    translation = [box_center[i] - centroid[i] for i in range(3)]

    # Translate all atoms
    for i in range(num_atoms):
        pos = conf.GetAtomPosition(i)
        new_pos = (pos.x + translation[0], pos.y + translation[1], pos.z + translation[2])
        conf.SetAtomPosition(i, new_pos)

    # Write translated molecule
    writer = Chem.SDWriter(output_sdf)
    writer.write(mol)
    writer.close()

In [7]:
def generate_ligand_files(smiles, pH, ligandPath,id, box_center):
    #generate_ligand_files(smiles, pH, f"{protein_name}/ligands/{i}",box_center)
    ligandFilename = f"{ligandPath}/base/{id}.pdbqt"
    ligandSDF = f"{ligandPath}/scrubbed/{id}_scrubbed.sdf"
    try:
        subprocess.run(["scrub.py", smiles, "-o", ligandSDF, "--ph", pH, "--skip_acidbase", "--skip_tautomer"], check=True,stdout=subprocess.DEVNULL)
    except:
        return False
    if not os.path.exists(ligandSDF) or os.path.getsize(ligandSDF) == 0:
      #print(f"Ligand {id} failed to scrub")
      return False
    translate_ligand_to_box_center(ligandSDF, f"{ligandSDF}", box_center)
    try:
        subprocess.run(["mk_prepare_ligand.py", "-i", f"{ligandSDF}", "-o", ligandFilename], check=True, stdout=subprocess.DEVNULL)
    except:
        return False
    if not os.path.exists(ligandFilename) or os.path.getsize(ligandFilename) == 0:
      #print(f"Ligand {id} failed to prepare")
      return False
    return True

In [8]:
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as fnull:
        old_stdout = os.dup(1)
        old_stderr = os.dup(2)
        os.dup2(fnull.fileno(), 1)
        os.dup2(fnull.fileno(), 2)
        try:
            yield
        finally:
            os.dup2(old_stdout, 1)
            os.dup2(old_stderr, 2)
            os.close(old_stdout)
            os.close(old_stderr)

def perform_ligand_docking(ligand_id,receptor_name, center, box_size):
  v = Vina(sf_name='vina')
  receptor_filename = f"{receptor_name}/{receptor_name}.pdbqt"
  with suppress_output(): 
    v.set_receptor(receptor_filename)

  ligand_filename = f"{receptor_name}/ligands/base/{ligand_id}.pdbqt"
  v.set_ligand_from_file(ligand_filename)
  with suppress_output():
    v.compute_vina_maps(center=center, box_size=box_size)

  ligand_filename_out = f"{receptor_name}/ligands/out/{ligand_id}_out"
  ligand_name = ligand_filename.replace(".pdbqt", "")
  with suppress_output(): 
    energy_minimized = v.optimize()

  # Dock the ligand
  with suppress_output():  
    v.dock(exhaustiveness=8, n_poses=20)
  v.write_poses(f'{ligand_filename_out}.pdbqt', n_poses=1, overwrite=True)
  subprocess.run(["mk_export.py", f"{ligand_filename_out}.pdbqt", "-s", f"{ligand_filename_out}.sdf"],
    stdout=subprocess.DEVNULL)
  energies = v.energies()
  return energies[0][0]

In [9]:
def single_docking(smiles, protein_name, pH, i, box_center, box_size):
  has_ligand_file = generate_ligand_files(smiles, pH, f"{protein_name}/ligands",i, box_center)
  if not has_ligand_file:
    return None
  try:
    energy = perform_ligand_docking(i,protein_name, box_center, box_size)

  except RuntimeError as e:
      error_msg = str(e)
      if "ligand is outside the grid box" in error_msg:
          print(f"Docking skipped for ligant {i}: ligand outside the grid box.")
          energy = None
      else:
          # Re-raise unexpected RuntimeErrors
          raise
  return energy

def batch_docking(smiles_batch,receptor,pH=7,vanilla_smiles="COC1=C(C=CC(=C1)C=O)O"):
  pH = str(pH)
  # Prepare protein
  protein_name = receptor.name
  lit_ref_energy = receptor.ref_energy
  box_size = receptor.box_size
  box_center = receptor.box_center

  os.makedirs(f"{protein_name}/ligands", exist_ok=True)
  os.makedirs(f"{protein_name}/ligands/base", exist_ok=True)
  os.makedirs(f"{protein_name}/ligands/out", exist_ok=True)
  os.makedirs(f"{protein_name}/ligands/scrubbed", exist_ok=True)
  generate_receptor_file(f"{protein_name}/{protein_name}.pdb", f"{protein_name}/{protein_name}", box_size, box_center)

  # Calculate reference energy
  ref_energy = single_docking(vanilla_smiles, protein_name, pH, "reference", box_center, box_size)
  print(61*"-")
  print("Referance state energy:")
  print(f"Literature: {lit_ref_energy} kcal")
  print(f"Estimated: {ref_energy} kcal")
  print(61*"-")
  energies = []
  print(f"{'it':<4} {'SMILES':<40} {'Energy [kcal]':>15}")
  for i,smiles in enumerate(smiles_batch):

    energy = single_docking(smiles, protein_name, pH, i, box_center, box_size)
    energies.append(energy)
    energy_str = f"{energy:>15.3f}" if isinstance(energy, (int, float)) else f"{'FAILED':>15}"
    print(f"{i:<4} {smiles:<40} {energy_str:>15}")
  assert len(energies) == len(smiles_batch)
  print(61*"-")
  df = pd.DataFrame({"smiles":smiles_batch,f"{protein_name} energies [kcal]":energies})
  reference_df = pd.DataFrame({"Receptor":[protein_name],f"Literature ref. energy [kcal]": [lit_ref_energy],f"Estimated ref. energy [kcal]": [ref_energy]})

  return df, reference_df


In [10]:
class Receptor():
  def __init__(self, name, min_lst, max_lst, url, ref_energy,amino_acids, from_folk_ntnu=False):
    self.name = name
    self.min_lst = min_lst
    self.max_lst = max_lst
    self.url = url
    self.ref_energy = ref_energy
    self.calc_bounding_box(self.min_lst, self.max_lst)
    self.from_folk_ntnu = from_folk_ntnu
    self.amino_acids = sorted(amino_acids)
    self.download()

  def download(self):

    url = self.get_url()

    response = requests.get(url)
    file_Path = f'{self.name}/{self.name}.pdb'
    os.makedirs(self.name, exist_ok=True)
    if response.status_code == 200:
      with open(file_Path, 'wb') as file:
          file.write(response.content)
      print(f'{self.name}.pdb downloaded successfully')
    else:
      print(f'Failed to download file {self.name}.pdb')

  def get_url(self):
    if self.from_folk_ntnu:
      return f"https://pauljg.folk.ntnu.no/olfactory_receptors/{self.name}.pdb"
    return self.url

  def set_download_host(self, from_folk_ntnu):
    self.from_folk_ntnu = from_folk_ntnu


  def perform_docking(self, smiles_batch,pH=7,vanilla_smiles="COC1=C(C=CC(=C1)C=O)O"):
    self.df, self.reference_df = batch_docking(smiles_batch,self,pH=pH,vanilla_smiles=vanilla_smiles)
    self.df.to_csv(f"{self.name}/{self.name}_docking_energies.csv")
    self.reference_df.to_csv(f"{self.name}/{self.name}_reference_energies.csv")
    return self.df,self.reference_df

  def get_docking_results(self):
    if hasattr(self, 'df'):
      return df
    else:
      raise AttributeError("No docking results available. Perform docking first.")

  def calc_bounding_box(self,min_lst, max_lst):
    [[x_min, y_min, z_min], [x_max, y_max, z_max]] = [min_lst,max_lst]

    self.box_center = ((x_min + x_max)/2, (y_min + y_max)/2, (z_min + z_max)/2)
    self.box_size = (x_max - x_min, y_max - y_min, z_max - z_min)
    return self.box_size, self.box_center

  def get_pyMOL_commands(self):
    """
    select <selction_name>, resi <amino_acid_ids>
    get_extent <selction_name>
    """
    acid_selctor = "+".join(map(str, self.amino_acids))
    return [
        f"select active_site, resi {acid_selctor}",
        f"get_extent active_site"
        ]

  def print_pyMOL_commands(self):
    print(f"PyMOL commands for {self.name}")
    command1, command2 = self.get_pyMOL_commands()
    print(command1)
    print(command2)
    print()




In [11]:
def perform_docking(receptors, smiles_batch,pH=7,vanilla_smiles="COC1=C(C=CC(=C1)C=O)O"):
  combined_df = pd.DataFrame({"SMILES":smiles_batch})
  lit_ref_energys = []
  ref_energys = []
  for receptor in receptors:
    print(f"Performing docking for {receptor.name}")
    df,reference_df = receptor.perform_docking(smiles_batch,pH=pH,vanilla_smiles=vanilla_smiles)
    combined_df[receptor.name] = df[f"{receptor.name} energies [kcal]"]
    lit_ref_energys.append(reference_df[f"Literature ref. energy [kcal]"][0])
    ref_energys.append(reference_df[f"Estimated ref. energy [kcal]"][0])
    print()
  combined_ref_df = pd.DataFrame({"Receptor":[receptor.name for receptor in receptors],f"Literature": lit_ref_energys,f"Estimated": ref_energys})
  combined_df.to_csv("combined_docking_energies.csv")
  combined_ref_df.to_csv("combined_reference_energies.csv")
  return combined_df, combined_ref_df



### Visualization functions

In [12]:
# Visualization with py3Dmol
def Receptor3DView(receptorPDB = None, boxPDB = None, ligmol = None):

    view = py3Dmol.view()
    view.setBackgroundColor('white')

    #view.addModel(open("protein.box.pdb", 'r').read(),'pdb')
    view.addModel(open(boxPDB, 'r').read(),'pdb')
    view.addStyle({'stick': {}})
    view.zoomTo()

    #view.addModel(open("OR1L3_cleaned.pdb", 'r').read(),'pdb')
    view.addModel(open(receptorPDB, 'r').read(),'pdb')
    view.addStyle({'cartoon': {'color':'spectrum', 'opacity': 0.5}})

    if ligmol is not None:
      mblock = Chem.MolToMolBlock(ligmol)
      view.addModel(mblock, 'mol')
      #view.addModel(open(ligPDB, 'r').read(), 'pdb')
      view.addStyle({'hetflag': True}, {'stick': {}})

    return view



In [13]:
def batch_visualization(index, receptor):
  protein_file = receptor.name

  # Load molecule from SDF
  supplier = Chem.SDMolSupplier(f"{protein_file}/ligands/out/{index}_out.sdf")
  mol = supplier[0]  # take first molecule

  Receptor3DView(receptorPDB=f"{protein_file}/{protein_file}.pdb", boxPDB=f"{protein_file}/{protein_file}.box.pdb", ligmol = mol).show()


In [14]:


# Visualization with py3Dmol
def Receptor3DView2(receptorPDB = None, boxPDB = None, ligmol = None, refmol=None):

    view = py3Dmol.view()
    view.setBackgroundColor('white')

    #view.addModel(open("protein.box.pdb", 'r').read(),'pdb')
    view.addModel(boxPDB,'pdb')
    view.addStyle({'stick': {}})
    view.zoomTo()

    #view.addModel(open("OR1L3_cleaned.pdb", 'r').read(),'pdb')
    view.addModel(receptorPDB,'pdb')
    view.addStyle({'cartoon': {'color':'spectrum', 'opacity': 0.5}})

    if ligmol is not None:
      mblock = Chem.MolToMolBlock(ligmol)
      view.addModel(mblock, 'mol')
      #view.addModel(open(ligPDB, 'r').read(), 'pdb')
      view.addStyle({'hetflag': True}, {'stick': {}})



    if refmol is not None:
      refmblock = Chem.MolToMolBlock(refmol)
      view.addModel(refmblock, 'mol')
      view.addStyle({'model': -1}, {"stick": {'color': 'lightblue', 'opacity': 0.6}})
    return view

#
def batch_visualization2(ligand_smiles, protein_name,smiles_batch,proteins, mols,refmols,protein_pbds,box_pbds):
  # Find the protein index from name
  protein_idx = proteins.index(protein_name)
  # Find ligand index from SMILES string
  ligand_idx = smiles_batch.index(ligand_smiles)


  # Load molecule from SDF
  #supplier = Chem.SDMolSupplier(f"{protein_file}/ligands/out/{index}_out.sdf")
  #mol = supplier[0]  # take first molecule
  #ref_mol = Chem.SDMolSupplier(f"{protein_file}/ligands/out/reference_out.sdf")[0]
  mol = mols[protein_idx][ligand_idx]
  refmol=refmols[protein_idx][0]
  Receptor3DView2(receptorPDB = protein_pbds[protein_idx], boxPDB = box_pbds[protein_idx], ligmol = mol, refmol=refmol).show()
  #Receptor3DView(receptorPDB=f"{protein_file}/{protein_file}.pdb", boxPDB=f"{protein_file}/{protein_file}.box.pdb", ligmol = mol).show()

def load_mol_safely(path):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        print(f"Corrupt {path} file. Molecule has been replaced with He")
        return Chem.MolFromSmiles("[He]")
    return Chem.SDMolSupplier(path)[0]

def interactive_batch_visualization(smiles_batch, receptors):
  refmols = [[Chem.SDMolSupplier(f"{r.name}/ligands/out/reference_out.sdf")[0]] for r in receptors]
  mols = [[load_mol_safely(f"{r.name}/ligands/out/{i}_out.sdf") for i in range(len(smiles_batch))] for r in receptors]
  protein_pbds = [open( f"{r.name}/{r.name}.pdb", 'r').read()  for r in receptors]
  box_pbds = [open(f"{r.name}/{r.name}.box.pdb", 'r').read() for r in receptors]
  # Launch interactive viewer
  proteins = [r.name for r in receptors]
  protein_dropdown = Dropdown(options=proteins, description='Protein:')
  ligand_dropdown = Dropdown(options=smiles_batch, description='Ligand:')
  #interact(lambda ligand_idx, protein_idx: batch_visualization2(ligand_idx,protein_idx, mols,refmols,protein_pbds,box_pbds),
   #        ligand_idx=ipywidgets.IntSlider(min=0, max=len(smiles_batch)-1, step=1),
   #       protein_idx=ipywidgets.IntSlider(min=0, max=len(receptors)-1, step=1)
    #       )
  interact(lambda ligand_smiles, protein_name: batch_visualization2(ligand_smiles,protein_name,smiles_batch,proteins, mols,refmols,protein_pbds,box_pbds),
           protein_name= protein_dropdown, ligand_smiles= ligand_dropdown)
           #ligand_idx=ipywidgets.IntSlider(min=0, max=len(smiles_batch)-1, step=1),
          #protein_idx=ipywidgets.IntSlider(min=0, max=len(receptors)-1, step=1)

  return

# Use

This code estimates binding energies of a set of ligands to different proteins. It is based on AutoDock Vina. AutoDock Vina utilizes **.pdb** files which have to be aquired for each protein. The protein structures used in this notebook are retrived from [AlphaFoldDB](https://alphafold.ebi.ac.uk/). The structure of one of the OR1L3 protein for example can be downloaded [here](https://alphafold.ebi.ac.uk/entry/Q8NH93). OR1L3 an active odor receptor for vanillin. This notebooks uses structural information for 10 such molecules. The region containing the active site has to be determined for each protein to perform docking. The amino acids relevant for the active sides used are described in this [paper](https://www.sciencedirect.com/science/article/pii/S002364382400584X).

The **.pdb** files can be opened in a tool like PyMOL to determine the bounding box coordinates of the active sides. This is done by running the following commands after opening the file for each protein.

```
select active_site, resi 259+258+255+263+277+109+207+199+159+203+182
get_extent active_site
```

The numbers in the first command correspond to amino acid indexes relevant for the active site in OR1L3 and can be replaced when the code is used for different proteins.

The active region for the OR1L3 has been found to be contained in the box described by the code below.

```
 cmd.extent: min: [  -2.083, -14.006, -13.817]
 cmd.extent: max: [  18.630,  -1.134,   6.993]
```

The information about the active side and the binding energy was extracted from the [paper](https://www.sciencedirect.com/science/article/pii/S002364382400584X) and was stored in a list containing `Receptor` objects. The receptor objects do also contain the url to the corresponding **.pdb** file. These files can alternativly also be downloaded from [folk.ntnu.no/pauljg](https://pauljg.folk.ntnu.no/olfactory_receptors/). This was done to ensure access to the file if AlphaFold removes the files. Initializing the receptor objects automatically downloads the corresponding **.pdb** files.

In [15]:
# https://www.sciencedirect.com/science/article/pii/S002364382400584X
from_folk_ntnu=True # Toggle download host to download files from folk.ntnu.no/pauljg. Activate if the alphafold link stops working.
RECEPTORS = [
    Receptor("OR1L3", [  -2.083, -14.006, -13.817], [  18.630,  -1.134,   6.993], "https://alphafold.ebi.ac.uk/files/AF-Q8NH93-F1-model_v4.pdb", -6.518, [182,259,258,255,263,277,109,207,199,159,203],from_folk_ntnu=from_folk_ntnu),
    Receptor("OR2G2", [  -6.776, -11.795, -10.361], [  12.779,   8.849,   6.585], "https://alphafold.ebi.ac.uk/files/AF-Q8NGZ5-F1-model_v4.pdb", -5.108, [181,183,184,76,286,254,209,210,115,262,258,112,111,108,107],from_folk_ntnu=from_folk_ntnu),
    Receptor("OR2J2", [  -3.834, -12.908, -10.161], [  18.039,   1.219,   8.826], "https://alphafold.ebi.ac.uk/files/AF-O76002-F1-model_v4.pdb", -5.376, [208,207,204,203,200,160,182,183,106,260,109,110,256,252,113],from_folk_ntnu=from_folk_ntnu),
    Receptor("OR2M4", [  -5.889,  -3.211,  -5.366], [  13.720,  15.215,  10.275], "https://alphafold.ebi.ac.uk/files/AF-Q96R27-F1-model_v4.pdb", -5.951, [104,108,111,278,279,281,282,283,73,74,77],from_folk_ntnu=from_folk_ntnu),
    Receptor("OR2T10", [  -5.305,  -1.740,  -3.296], [   9.105,  14.118,  10.838], "https://alphafold.ebi.ac.uk/files/AF-Q8NGZ9-F1-model_v4.pdb", -5.720, [254,251,250,247,282,111,277,278],from_folk_ntnu=from_folk_ntnu),
    Receptor("OR2T34", [  -6.223,  -3.826, -16.302], [   8.484,  12.286,   1.332], "https://alphafold.ebi.ac.uk/files/AF-Q8NGX1-F1-model_v4.pdb", -4.992, [185,187,109,113,259,260,263,283,280,279,276],from_folk_ntnu=from_folk_ntnu),
    Receptor("OR5AC2", [  -1.415,  -7.356,  -4.174], [  17.525,  13.128,  13.700], "https://alphafold.ebi.ac.uk/files/AF-Q9NZP5-F1-model_v4.pdb", -5.441, [257,253,106,280,110,111,113,114,209,208,157,205,261,201],from_folk_ntnu=from_folk_ntnu),
    Receptor("OR10G3", [  -5.200, -12.383,  -6.671], [  14.032,   3.703,   8.274], "https://alphafold.ebi.ac.uk/files/AF-Q8NGC4-F1-model_v4.pdb", -6.098, [252,113,110,109,106,208,207,204,203,277,278,256],from_folk_ntnu=from_folk_ntnu),
    Receptor("OR10G4", [  -3.106, -15.073,  -8.889], [  15.238,   6.544,   6.971], "https://alphafold.ebi.ac.uk/files/AF-Q8NGN3-F1-model_v4.pdb", -6.183, [254,111,108,107,104,103,206,205,202,154,158,71,258],from_folk_ntnu=from_folk_ntnu),
    Receptor("OR10G7", [  -5.632, -12.023,  -6.291], [  14.197,   7.193,   8.432], "https://alphafold.ebi.ac.uk/files/AF-Q8NGN6-F1-model_v4.pdb", -6.030, [206,205,202,154,155,111,108,107,104,103,275,71,254,251,250],from_folk_ntnu=from_folk_ntnu),
    ]

OR1L3.pdb downloaded successfully
OR2G2.pdb downloaded successfully
OR2J2.pdb downloaded successfully
OR2M4.pdb downloaded successfully
OR2T10.pdb downloaded successfully
OR2T34.pdb downloaded successfully
OR5AC2.pdb downloaded successfully
OR10G3.pdb downloaded successfully
OR10G4.pdb downloaded successfully
OR10G7.pdb downloaded successfully


The `Receptor` objects can also be used to generate the PyMOL commands for infoformation about the active side, by running the `print_pyMOL_commands` method.  

In [16]:
for receptor in RECEPTORS:
  receptor.print_pyMOL_commands()

PyMOL commands for OR1L3
select active_site, resi 109+159+182+199+203+207+255+258+259+263+277
get_extent active_site

PyMOL commands for OR2G2
select active_site, resi 76+107+108+111+112+115+181+183+184+209+210+254+258+262+286
get_extent active_site

PyMOL commands for OR2J2
select active_site, resi 106+109+110+113+160+182+183+200+203+204+207+208+252+256+260
get_extent active_site

PyMOL commands for OR2M4
select active_site, resi 73+74+77+104+108+111+278+279+281+282+283
get_extent active_site

PyMOL commands for OR2T10
select active_site, resi 111+247+250+251+254+277+278+282
get_extent active_site

PyMOL commands for OR2T34
select active_site, resi 109+113+185+187+259+260+263+276+279+280+283
get_extent active_site

PyMOL commands for OR5AC2
select active_site, resi 106+110+111+113+114+157+201+205+208+209+253+257+261+280
get_extent active_site

PyMOL commands for OR10G3
select active_site, resi 106+109+110+113+203+204+207+208+252+256+277+278
get_extent active_site

PyMOL commands for O

A batch of smiles strings is also required for evaluating docking for a set of molecules.

In [17]:
results_df = pd.read_csv("first_configuration_intermediate_results.csv").iloc[:500]

SMILES_BATCH = results_df["SMILES"]




The batches of SMILES strings and receptors are then passed to the docking function which returns a dataframe containing the best docking enery found for a ligant and its SMILES representation for each receptor. It also returns a second dataframe containing vanillins predicted and [literature](https://www.sciencedirect.com/science/article/pii/S002364382400584X) binding energies for reference. The dataframes are saved as **.csv** files. Molecules unable fit inside the active site box for a protein are disgarded and receive a energy value of None assuming that they wont dock since they are sterically blocked from entering.

In [ ]:
%%time
combined_df, combined_ref_df = perform_docking(RECEPTORS, SMILES_BATCH,pH=7,vanilla_smiles="COC1=C(C=CC(=C1)C=O)O")

Performing docking for OR1L3
-------------------------------------------------------------
Referance state energy:
Literature: -6.518 kcal
Estimated: -5.478 kcal
-------------------------------------------------------------
it   SMILES                                     Energy [kcal]
0    CC=CCSCC(C)=CC                                    -4.562
1    CCCC(=CCS)OC                                      -4.251
2    CCCCC(C)C=C(C)CCCN                                -6.304
3    C=C(CC)CCSC                                       -4.241
4    C=C(S)C(=CCC(=O)OC)C(C)C(C)C                      -5.905
5    CCCC(=CN)C(C)=O                                   -5.320
6    C=C(C)C(C)C1=COC=CC=C=C1                          -5.015
7    CC(=O)C(C)C=C(C)CC(C)C                            -6.074
8    CC=CC(=CCCC)SC                                    -4.941
9    CCC=C(S)CCC(=O)CC                                 -5.207
10   CC=C(CC=CCC)OC(=O)CC                              -6.055
11   CCSC(C)C=CC(C)CC           

{'INITIAL_COORDS': 105, 'FIRST_MINIMIZATION': 215}


212  CCS1(C)(C)(C)CC1(C)C                              FAILED
213  CC=CC=COC(=O)C(C=O)=CCC                           -6.155
214  CCCC(CC)CC=CCC(C)CC                               -6.489
215  C=C(C)SC(C)CCCC                                   -5.077
216  C=CC(CC)(CO)CCO                                   -5.418
217  C=CC(C)=C[SH]=CC=CC                               -4.860
218  C=COC(=O)C(C)CC(=C)C                              -5.447
219  CCC(=O)CC=C(C)C                                   -5.163
220  CCCCC=COCCO                                       -5.121
221  CCCOC(O)(CC)CC                                    -5.041
222  CCCC(C)(C)C(Cl)CC(C)C(C)=O                        -6.655
223  CC(C)(CS)C(C=O)C1CO1                              -5.233
224  CCC(CC)[C+](C)C(C)C1=CC=C1C                       -6.768
225  CCC(=C(C)C)C(O)C=CC=O                             -5.682
226  C=CC=C(CCC)CC(C)=O                                -5.526
227  C=C1N=C(SC)OCC1CC                                 -5.454
228  CCC

{'INITIAL_COORDS': 63, 'LINEAR_DOUBLE_BOND': 187}


242  C=C=C(C=COC)C1CCC1                                FAILED
243  C=C(C)C(C=COC(C)CC)CC                             -5.873
244  C=C(C)C=CC1(N=CC)CN(C)C1                          -5.905
245  C=C(C)SCCCC                                       -4.300
246  C=NC=CC(=C)C                                      -4.505


...

341  C=C1C=C(C)CCC1=CC(C)[Na]                          FAILED
342  CCOC(CCCO)CCC(C)C                                 -5.402
343  CCCCC1CC1CCO                                      -4.564
344  C=CC=CC(=O)C(C)(O)CC                              -5.381
345  C=CSC(C)C(CC)CC                                   -4.524
346  CCCC(CC(C)=O)C(C)=C1C=C1C                         -6.093
347  C=COC(C(=C)CCC)=C(C)C=C                           -6.049
348  CC1=CCC(=CC(C)(C)CC2=CC=C=C3CC(C2)O3)C1           -5.196
349  C=CC=C1C=CC(=C)CC1                                -6.710
350  CC=C(CCC)C(C)=O                                   -5.445
351  CC(C)=C(C)C1=C2C(=C(OCC=O)C2(C)C)C1               -6.452
352  CCCC(CC)=C(O)OCC                                  -5.174
353  CC=C1C=CCOC=CC(=O)C1                              -3.853


{'ETK_MINIMIZATION': 200}


354  Cc1cc2cc3c1CC(=C3)C2=O                            FAILED
355  C=C(C)C=C(C)C=CC(=O)O                             -5.608
356  CCCC[SH](S)CN                                     -4.112
357  CCC1=C=[SH]C(CC)C(C)O1                            -5.146
358  C=CC(C=C)SC=CCC                                   -4.592
359  CC=CC=C(OCC)S1=CC1SOC                             -4.074
360  C=CC1SCCC1=C=C=O                                  -4.401
361  CCCC(=O)C(N)C=CC(C)C                              -5.443
362  CC=C(CC)C(S)CC                                    -4.780
363  C=C1CC(CC)C2C(=O)C(CC)CC2C1                       -5.133
364  CCC(CC[SH]=C(C)C)CC(C)O                           -5.488
365  C=CN=C(O)C(CC)CCC                                 -5.082
366  CCOC1=CC(C)C(C)CCC1                               -3.916
367  CCCCNC1=CC=CC1                                    -5.243
368  C=C(C)C=CC=C(C)CCC=C(C)C                          -5.715
369  COC1=C(O)C[C](OC(C)C)C1=O                         -5.000
370  CCC

atom number 5 has None type, mol name: 
atom number 0 has non finite charge, mol name: , charge: nan
atom number 1 has non finite charge, mol name: , charge: nan
atom number 2 has non finite charge, mol name: , charge: nan
atom number 3 has non finite charge, mol name: , charge: nan
atom number 4 has non finite charge, mol name: , charge: nan
atom number 5 has None type, mol name: 
atom number 5 has non finite charge, mol name: , charge: nan
atom number 6 has non finite charge, mol name: , charge: nan
atom number 7 has non finite charge, mol name: , charge: nan
atom number 8 has non finite charge, mol name: , charge: nan
atom number 9 has non finite charge, mol name: , charge: nan
atom number 10 has non finite charge, mol name: , charge: nan
atom number 11 has non finite charge, mol name: , charge: nan
atom number 12 has non finite charge, mol name: , charge: nan
atom number 13 has non finite charge, mol name: , charge: nan
atom number 14 has non finite charge, mol name: , charge: nan


455  CC1=C2C(C[Na])C=CC(=CC=O)C=CC=C1C2(C)C            FAILED
456  CCC1CCC=CC([SH]=O)C1                              -3.874
457  C=C(C=C(C)OC=O)CCO                                -5.387
458  CCC=CC1(C2(C)C=CCC2)[C]C=C1                       -6.518
459  CCC1CC(C)C(CCC=O)C1                               -5.450
460  CCC(C)SC(C)C                                      -4.406
461  CC=C(C)C=[SH]CC                                   -4.368
462  CCC(N)CCCC(C)O                                    -4.958
463  C=CCC(=O)C=C=CC(C)=CC=C(CC)CC                     -6.144
464  CCCCC=C(C)C(C)C=O                                 -5.864
465  C=C(C)CC1CCCCCC(OC)O1                             -4.302
466  CC=CCOC(=O)C(C)CCCC(C)C                           -5.800
467  CC1CC2C=C(CS1)CC2C                                -2.558
468  C=S(C=CC)CC(C)(C)C=COCSS                          -4.767
469  CSC(S)SC                                          -2.869
470  CC=C(C)CC(CC)SS                                   -4.589


The data contained in the two generated DataFrames is shown below:

In [ ]:
combined_df

In [ ]:
combined_ref_df

We can visualize the docking of molecules with two different functions. `interactive_batch_visualization` creates an interactive plot were we can pick a ligand from its smiles representation and a protein by name. Vanillin is also shown slightly transparently in light blue to visualize if a molecule is binding at aslightly different position compared to vanillin.   

In [ ]:
interactive_batch_visualization(list(SMILES_BATCH), RECEPTORS)

The `batch_visualization` function takes an SMILES index **i** and a `Receptor` object  as shown below.

In [ ]:
batch_visualization(0, RECEPTORS[0])

# Analyze results

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

Heatmap to visualize binding energies of different molecules for different receptors. A very low (negative) value indicates a high affinity.

In [ ]:
sns.heatmap(combined_df.iloc[:25].set_index("SMILES"), cmap="crest", annot=True)
plt.xlabel("Olfactory Receptor")

In [ ]:
sns.heatmap(combined_df.iloc[25:50].set_index("SMILES"), cmap="crest", annot=True)
plt.xlabel("Olfactory Receptor")

In [ ]:
sns.heatmap(combined_df.iloc[50:75].set_index("SMILES"), cmap="crest", annot=True)
plt.xlabel("Olfactory Receptor")

In [ ]:
sns.heatmap(combined_df.iloc[75:].set_index("SMILES"), cmap="crest", annot=True)
plt.xlabel("Olfactory Receptor")

In [ ]:
combined_df["SMILES"][2]

In [ ]:
results_df["R(x)"][2]

In [ ]:
results_df["vanilla"][2]

In [ ]:
results_df["R(x)"][10],results_df["vanilla"][10]

In [ ]:
combined_df["SMILES"][20]

In [ ]:
results_df["R(x)"][20],results_df["vanilla"][20]

Box plot to show the binding energy distibutions of the generated molecules compared to the binding affinity of vanillin (Both predicted and from literature)

In [ ]:
ax = sns.boxplot(data=combined_df.set_index("SMILES"), orient="h")
ax.set_ylabel("Olfactory Receptor")
ax.set_xlabel("Binding energy")
ax.scatter(combined_ref_df["Literature"],combined_ref_df["Receptor"], label="Vanillin literature", zorder=100,marker="p",facecolors='none', edgecolors=(0,0,0,0.6))#, color=(0,0,0,0.8) )
ax.scatter(combined_ref_df["Estimated"],combined_ref_df["Receptor"], label="Vanillin estimated", zorder=100,marker="D",facecolors='none', edgecolors=(0,0,0,0.6))#, color=(0,0,0,0.8) )
ax.legend()
sns.move_legend(ax, "upper right")

Another boxplot to show how well different molecules bind to different proteins and to determine if some molecules have high variance. The mean binding energies for vanillin are also plotted to see which molecules bind similary

In [ ]:
df_long = combined_df.iloc[:50].melt(id_vars='SMILES', var_name='Protein', value_name='BindingEnergy')
fig, ax = plt.subplots(figsize=(4, 10))
ax = sns.boxplot(x='BindingEnergy', y='SMILES', data=df_long,palette='tab10', hue="SMILES", legend=False, orient="h", ax=ax)
#plt.xticks(rotation=90)
ax.set_xlabel("Energy [kcal]")
ax.axvline(combined_ref_df["Literature"].mean(), linestyle="--", zorder=-2,color=(0,0,0,0.8),label="Literature mean")
ax.axvline(combined_ref_df["Estimated"].mean(), linestyle=":", zorder=-2, color=(0,0,0,0.8),label="Estimated mean")

plt.legend()
sns.move_legend(ax, "upper right")

In [ ]:
df_long = combined_df.iloc[25:50].melt(id_vars='SMILES', var_name='Protein', value_name='BindingEnergy')

ax = sns.boxplot(x='BindingEnergy', y='SMILES', data=df_long,palette='tab10', hue="SMILES", legend=False, orient="h")
#plt.xticks(rotation=90)
ax.set_xlabel("Energy [kcal]")
ax.axvline(combined_ref_df["Literature"].mean(), linestyle="--", zorder=-2,color=(0,0,0,0.8),label="Literature mean")
ax.axvline(combined_ref_df["Estimated"].mean(), linestyle=":", zorder=-2, color=(0,0,0,0.8),label="Estimated mean")

plt.legend()
sns.move_legend(ax, "upper right")

In [ ]:
df_long = combined_df.iloc[50:75].melt(id_vars='SMILES', var_name='Protein', value_name='BindingEnergy')

ax = sns.boxplot(x='BindingEnergy', y='SMILES', data=df_long,palette='tab10', hue="SMILES", legend=False, orient="h")
#plt.xticks(rotation=90)
ax.set_xlabel("Energy [kcal]")
ax.axvline(combined_ref_df["Literature"].mean(), linestyle="--", zorder=-2,color=(0,0,0,0.8),label="Literature mean")
ax.axvline(combined_ref_df["Estimated"].mean(), linestyle=":", zorder=-2, color=(0,0,0,0.8),label="Estimated mean")

plt.legend()
sns.move_legend(ax, "upper right")

In [ ]:
df_long = combined_df.iloc[75:].melt(id_vars='SMILES', var_name='Protein', value_name='BindingEnergy')

ax = sns.boxplot(x='BindingEnergy', y='SMILES', data=df_long,palette='tab10', hue="SMILES", legend=False, orient="h")
#plt.xticks(rotation=90)
ax.set_xlabel("Energy [kcal]")
ax.axvline(combined_ref_df["Literature"].mean(), linestyle="--", zorder=-2,color=(0,0,0,0.8),label="Literature mean")
ax.axvline(combined_ref_df["Estimated"].mean(), linestyle=":", zorder=-2, color=(0,0,0,0.8),label="Estimated mean")

plt.legend()
sns.move_legend(ax, "upper right")

In [ ]:
sns.pairplot(combined_df)

In [ ]:
#import random

In [ ]:
#reward_metric_df = pd.DataFrame({"SMILES":combined_df["SMILES"]})
#reward_metric_df["Binding energy [kcal]"]=list(combined_df.set_index("SMILES").mean(axis=1))
#reward_metric_df["Reward [-]"] = [random.random() for i in combined_df["SMILES"]]
#reward_metric_df["Vanilla Probabilty [-]"] = [random.random() for i in combined_df["SMILES"]]
#reward_metric_df

In [ ]:
len(combined_df)

In [ ]:
reward_metric_df = pd.DataFrame({"SMILES":combined_df["SMILES"]})
reward_metric_df["Binding energy [kcal]"]=list(combined_df.set_index("SMILES").mean(axis=1))
reward_metric_df["Reward [-]"] = results_df["R(x)"] #[random.random() for i in combined_df["SMILES"]]
reward_metric_df["Vanilla Probabilty [-]"] = results_df["vanilla"]#[random.random() for i in combined_df["SMILES"]]#reward_metric_df

In [ ]:
sns.pairplot(reward_metric_df)

In [ ]:
combined_df.keys()

In [ ]:
len(results_df)

In [ ]:
results_df.keys()

In [ ]:
df_cleaned = combined_df.copy()
df_cleaned["is_valid"] =results_df["is_valid"]
df_cleaned.head()

In [ ]:
docking_cols = df_cleaned.keys()[1:-1]
docking_cols

In [ ]:
import numpy as np

In [ ]:
df_cleaned.loc[~df_cleaned['is_valid'], docking_cols] = np.nan

In [ ]:
df_cleaned = df_cleaned[df_cleaned.keys()[:-1] ]
df_cleaned.head()

In [ ]:
ax = sns.boxplot(data=df_cleaned.set_index("SMILES"), orient="h")
ax.set_ylabel("Olfactory Receptor")
ax.set_xlabel("Binding energy")
ax.scatter(combined_ref_df["Literature"],combined_ref_df["Receptor"], label="Vanillin literature", zorder=100,marker="p",facecolors='none', edgecolors=(0,0,0,0.6))#, color=(0,0,0,0.8) )
ax.scatter(combined_ref_df["Estimated"],combined_ref_df["Receptor"], label="Vanillin estimated", zorder=100,marker="D",facecolors='none', edgecolors=(0,0,0,0.6))#, color=(0,0,0,0.8) )
ax.legend()
sns.move_legend(ax, "upper right")